In [ ]:
# Kiểm tra logs trong database
# Sử dụng Django shell: python manage.py shell
# Sau đó chạy:

# from django.db import connection
# with connection.cursor() as cursor:
#     cursor.execute("SELECT COUNT(*) FROM logs")
#     count = cursor.fetchone()[0]
#     print(f"Tổng số logs: {count}")
#     
#     cursor.execute("SELECT level, channel, message, context, created_at FROM logs ORDER BY created_at DESC LIMIT 10")
#     rows = cursor.fetchall()
#     for row in rows:
#         print(f"[{row[4]}] {row[0].upper()} - {row[1]}: {row[2]}")
#         if row[3]:
#             print(f"  Context: {row[3]}")

print("Chạy code trên trong Django shell để kiểm tra logs")

In [22]:
from vnstock import Company
from vnstock import Listing
from vnstock import Finance
company = Company(symbol = "TPB", source="TCBS")
company2 = Company(symbol = "TEG", source="VCI")
listing = Listing()
fin = finance = Finance(symbol="VTO", source="VCI")



In [23]:
company2.shareholders()


,id,share_holder,quantity,share_own_percent,update_date
0,88576397,Công ty Cổ phần Tập Đoàn Trường Thành Việt Nam,38334389,3.173000e-01,2025-02-11
1,88575296,Công ty Cổ Phần Đầu Tư Mernus,10921062,9.040000e-02,2024-07-24
2,88593178,Nguyễn Thị Thanh Hà,8725700,7.220000e-02,2025-02-28
3,88599691,Sermsang Power Corporation Public Company Limited,6041111,5.000000e-02,2024-01-29
4,88574996,Công ty Cổ phần Chứng khoán Sài Gòn - Hà Nội,1200000,3.710000e-02,2021-07-20
5,88594538,Nguyễn Tùng Lâm,997420,3.390000e-02,2025-02-11
6,88594038,Nguyễn Tích Phương,433000,2.890000e-02,2025-02-11
7,88598279,Phan Ngọc Anh Cương,1571984,2.400000e-02,2022-03-18
8,88591656,Nguyễn Thị Bích Liên,1101276,1.680000e-02,2023-02-10
9,88586349,Mạc Quang Huy,1976065,1.640000e-02,2025-02-11


In [9]:
# Kiểm tra cấu trúc bảng logs hiện tại
from django.db import connection
cursor = connection.cursor()

# Kiểm tra các cột trong bảng logs
cursor.execute("""
    SELECT column_name, data_type, is_nullable 
    FROM information_schema.columns 
    WHERE table_name = 'logs'
    ORDER BY ordinal_position
""")
columns = cursor.fetchall()
print("Cấu trúc bảng logs hiện tại:")
for col in columns:
    print(f"  {col[0]} - {col[1]} (nullable: {col[2]})")

print("\n" + "="*50)

# Drop và tạo lại bảng với cấu trúc đúng
cursor.execute("DROP TABLE IF EXISTS logs CASCADE")
print("Đã xóa bảng logs cũ")

cursor.execute("""
    CREATE TABLE logs (
        id BIGSERIAL PRIMARY KEY,
        level VARCHAR(20) NOT NULL,
        channel VARCHAR(50) NOT NULL,
        message TEXT NOT NULL,
        context JSONB DEFAULT '{}',
        extra JSONB DEFAULT '{}',
        environment VARCHAR(50),
        created_at TIMESTAMP WITH TIME ZONE DEFAULT NOW()
    );
    
    CREATE INDEX idx_logs_created_at ON logs(created_at);
    CREATE INDEX idx_logs_level ON logs(level);
    CREATE INDEX idx_logs_channel ON logs(channel);
    CREATE INDEX idx_logs_context_gin ON logs USING GIN(context);
""")
print("Đã tạo lại bảng logs với cấu trúc đúng")

# Test insert
cursor.execute("""
    INSERT INTO logs (level, channel, message, context, environment)
    VALUES (%s, %s, %s, %s, %s)
""", ['info', 'test', 'Test after recreate', '{"test": true}', 'local'])

print("Test insert thành công!")

# Kiểm tra
cursor.execute("SELECT COUNT(*) FROM logs")
count = cursor.fetchone()[0]
print(f"Số logs sau khi tạo lại: {count}")

SynchronousOnlyOperation: You cannot call this from an async context - use a thread or sync_to_async.